## StarTracker WCS solutions - 20-Feb-23

Craig Lage - Exploring moving to boresight locations.

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from lsst.obs.lsst.translators.latiss import AUXTEL_LOCATION
from lsst.obs.lsst.translators.lsst import SIMONYI_LOCATION

import lsst.afw.image as afwImage
from lsst.summit.utils.utils import starTrackerFileToExposure
from lsst.summit.utils.astrometry.anet import CommandLineSolver
from lsst.summit.utils.astrometry.utils import runCharactierizeImage

In [ ]:
solver = CommandLineSolver('/project/shared/ref_cats/astrometry_net/')

In [ ]:
cameras = [['Wide', 101], ['Narrow', 102]]
dates = [20230130]
dRAs = []
dDecs = []
for date in dates:
    #outfile = open(f"/scratch/cslage/starTracker/StarTracker_boresight_Diffs_{date}.txt", "w")
    #outfile.write("SeqNum\tRA(solve)\tDec(Solve)\tAz(Solve)\tEl(Solve)\tdeltaRA\tDeltaDec\n")

    for seqNum in range(150):
        RADecs = {}
        AltAzs = {}
        for [camera, num] in cameras:
            year = int(date/10000)
            month = int((date - 10000 * year)/100)
            day = int((date - 10000 * year - 100 * month))
            path = f"/project/GenericCamera/{num}/{year}/{month:02}/{day:02}/"
            try:    
                filename = path + f"GC{num}_O_{date}_{seqNum:06}.fits"
                exp = starTrackerFileToExposure(filename)
                results = runCharactierizeImage(exp, 4, 20)
                if camera == 'Narrow':
                    # Need to loosen percentageScaleError because header SECPIX is wrong
                    solverResults = solver.run(exp, results.sourceCat, percentageScaleError=60.0, isWideField=False)
                    #boresight = (1867.87, 974.58) # From Adrian
                    boresight = (1898.10, 998.47) # From assuming Wide is correct.
                if camera == 'Wide':
                    solverResults = solver.run(exp, results.sourceCat, isWideField=True)
                    boresight = (1560.85, 1257.151)
                newWcs = solverResults.wcs
                rms_error = solverResults.rmsErrorArsec
                #raSolve = newWcs.getSkyOrigin().getRa().asDegrees()
                #decSolve = newWcs.getSkyOrigin().getDec().asDegrees()

                boresight_RaDec = newWcs.pixelToSky(boresight[0], boresight[1])
                raSolve = boresight_RaDec.getRa().asDegrees()
                decSolve = boresight_RaDec.getDec().asDegrees()
                RADecs[camera] = [raSolve, decSolve]
            except:
                print(f"{camera}, {date}_{seqNum} Failed!")
                continue
            hdulist = pf.open(filename)
            header = hdulist[0].header
            raStart = header["RASTART"]
            decStart = header["DECSTART"]
            azHdr = (header["AZSTART"] + header["AZEND"]) / 2.0
            azHdr = azHdr % 360
            elHdr = (header["ELSTART"] + header["ELEND"]) / 2.0
            rotpa = header["ROTPA"]
            rot = newWcs.getRelativeRotationToWcs(exp.getWcs()).asDegrees()
            #time = Time(header["DATE-OBS"], scale='tai')
            time = Time((Time(header["DATE-BEG"], scale='tai').unix_tai + Time(header["DATE-END"], scale='tai').unix_tai)/2.0, format='unix_tai', scale='tai')
            altAz = AltAz(obstime=time, location=SIMONYI_LOCATION)
            skyLocation = SkyCoord(raSolve * u.deg, decSolve * u.deg)
            obsAltAz = skyLocation.transform_to(altAz)
            az = obsAltAz.az.deg
            az = az % 360
            el = obsAltAz.alt.deg
            AltAzs[camera] = [az, el]
            deltaAz = (azHdr - az) * 3600.0
            deltaEl = (elHdr - el) * 3600.0
            deltaRA = (raStart - raSolve) * 3600.0
            deltaDec = (decStart - decSolve) * 3600.0
            print(f"{camera}, {date}_{seqNum}, {raSolve:.5f}, {decSolve:.5f}, {az:.5f}, {el:.5f}")
        try:
            dRA = (RADecs['Wide'][0] - RADecs['Narrow'][0]) * 3600 * np.cos(decSolve)
            dDec = (RADecs['Wide'][1] - RADecs['Narrow'][1]) * 3600
            dRAs.append(dRA)
            dDecs.append(dDec)
            print(dRA, dDec)
            #outfile.write(f"{seqNum}\t{raSolve:.5f}\t{decSolve:.5f}\t{az:.5f}\t{el:.5f}\t{dRA:.1f}\t{dDec:.1f}\n")
        except:
            continue
    #outfile.close()
    

In [ ]:
# This calculates (for image 115)  what the narrow camera boresight would be if we translate the wide camera
# boresight into the narrow camera.
# Result = (1898.1, 998.47)

import lsst.afw.image as afwImage
from lsst.geom import SpherePoint
from lsst.geom import Angle as afwAngle

spherePoint = SpherePoint(afwAngle(RADecs['Wide'][0]*np.pi/180.0), afwAngle(RADecs['Wide'][1]*np.pi/180.0))
point = newWcs.skyToPixel(spherePoint)
print(point)

In [ ]:
dRAs2 = np.array(dRAs)
dDecs2 = np.array(dDecs)
RMS = np.sqrt(np.sum(dRAs2*dRAs2 + dDecs2*dDecs2)) / len(dRAs)

In [ ]:
%matplotlib inline
plt.subplot(1,1,1,aspect=1)
plt.title("StarTracker Boresight Deltas (Wide-Narrow) 20230130")
plt.scatter(dRAs, dDecs, marker='o', color='red', label='Wide boresight')
plt.scatter([0.0], [0.0], marker='X', color='green', s=100, label='Narrow boresight')
plt.legend()
plt.xlim(-150.0, 150.0)
plt.ylim(-150.0, 150.0)
plt.xlabel("DeltaRA (arcseconds)")
plt.ylabel("DeltaDec (arcseconds)")
plt.text(-100.0, 50.0, f"RMS offset = {RMS:.2f} arcseconds")
plt.savefig(f"/scratch/cslage/starTracker/StarTracker_Boresight_Diffs_Assumes_Wide_Correct_{date}.png")

In [ ]:
filename='/scratch/cslage/starTracker/StarTracker_boresight_Diffs_20230130.txt'
data = np.loadtxt(filename, skiprows=1)
Azs = []
Els = []
RAs = []
Decs = []
dRAs = []
dDecs = []
dist = []
for j in range(data.shape[0]):
    [seqNum,ra,dec,az,el,dRA,dDec] = data[j]
    RAs.append(ra)
    Decs.append(dec)
    Azs.append(az)
    Els.append(el)
    dRAs.append(dRA)
    dDecs.append(dDec)
    dist.append(np.sqrt(dRA*dRA + dDec*dDec))
    


In [ ]:
plt.subplots_adjust(hspace=0.5, wspace=0.5)
plt.subplot(2,2,1)
plt.title("DeltaRA vs Az")
plt.scatter(Azs, dRAs)
plt.subplot(2,2,2)
plt.title("DeltaRA vs El")
plt.scatter(Els, dRAs)
plt.subplot(2,2,3)
plt.title("DeltaDec vs Az")
plt.scatter(Azs, dDecs)
plt.subplot(2,2,4)
plt.title("DeltaDec vs El")
plt.scatter(Els, dDecs)
plt.savefig(f"/scratch/cslage/starTracker/Boresight_Diffs_vs AzEl_{date}.png")